<a href="https://colab.research.google.com/github/shakthi18/Onlineshpping/blob/main/MultiClassClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/alvinrindra/consumer-complaints


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56442 sha256=9f731a76bf0b7cd233e2cb133e3749ab6e7896afa436bf5d54f207c953c66ece
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import numpy as np
import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Consumer_Complaints.csv')

In [ ]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217
1,10/01/2016,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,10/05/2016,Closed with explanation,Yes,No,2141773
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No,2163100
3,06/08/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06/10/2014,Closed with explanation,Yes,Yes,885638
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760


In [ ]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [ ]:
data = df[['Consumer complaint narrative','Product']]
data = data.dropna()
data.head(100 )
     

,Consumer complaint narrative,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection
...,...,...
4736,I refinanced i year ago and the lender failed ...,Mortgage
4737,"XXXX account # XXXX I have insurance, I paid m...",Credit reporting
4743,I have XXXX hard inquiries from Safe Rent ( XX...,Mortgage
4744,Check out XXXXfor detailed descriptions on how...,Consumer Loan


In [ ]:
data.shape

(249830, 2)

In [ ]:
data.dtypes

Consumer complaint narrative    object
Product                         object
dtype: object

In [ ]:
data.Product.value_counts()

Debt collection                                                                 57324
Mortgage                                                                        41219
Credit reporting, credit repair services, or other personal consumer reports    37679
Credit reporting                                                                31593
Credit card                                                                     18842
Student loan                                                                    15495
Bank account or service                                                         14887
Consumer Loan                                                                    9473
Credit card or prepaid card                                                      7971
Checking or savings account                                                      4915
Vehicle loan or lease                                                            2065
Payday loan                                           

In [ ]:
data.Product.loc[(data['Product'] == "Credit reporting") | (data['Product'] == "Credit reporting, credit repair services, or other personal consumer reports") | 
         (data['Product'] == "Credit card") | (data['Product'] == "Credit card or prepaid card") | (data['Product'] == "Prepaid card")] = "Credit Card Related"

data.Product.loc[(data['Product'] == "Student loan") | (data['Product'] == "Consumer Loan") | 
         (data['Product'] == "Vehicle loan or lease") | (data['Product'] == "Payday loan") | (data['Product'] == "Payday loan, title loan, or personal loan")] = "Loan Related"

data.Product.loc[(data['Product'] == "Bank account or service") | (data['Product'] == "Checking or savings account") | 
         (data['Product'] == "Money transfer, virtual currency, or money service") | (data['Product'] == "Money transfers")] = "Bank Services"

data.Product.loc[(data['Product'] == "Debt collection") | (data['Product'] == "Mortgage") | 
         (data['Product'] == "Virtual currency") | (data['Product'] == "Other financial service") | (data['Product'] == "Prepaid card")] = "Others Services"

data.reset_index(inplace = True)
data.drop('index',axis='columns', inplace=True)

In [ ]:
data.Product.value_counts()

Others Services        98852
Credit Card Related    97535
Loan Related           30401
Bank Services          23042
Name: Product, dtype: int64

In [ ]:
data.head()

,Consumer complaint narrative,Product
0,I have outdated information on my credit repor...,Credit Card Related
1,I purchased a new car on XXXX XXXX. The car de...,Loan Related
2,An account on my credit report has a mistaken ...,Credit Card Related
3,This company refuses to provide me verificatio...,Others Services
4,This complaint is in regards to Square Two Fin...,Others Services


In [ ]:
def example_complain(index):
    example = data[data.index == index][['Consumer complaint narrative', 'Product']].values[0]
    if len(example) > 0:
        print(example[0])
        print()
        print('Category:', example[1])

In [ ]:
example_complain(10)

Without provocation, I received notice that my credit line was being decreased by nearly 100 %. My available credit was reduced from $ XXXX to XXXX ( the rough amount of my available balance ). 

When I called to question the change, I was provided a nob-descript response referencing my XXXX report. It was my understanding that under the FCRA I was entitled to a copy of this report, but was refused by Citi and have been given no further explanation. 

This is predatory in that it affects my utilization of credit, further subjecting me to increase in APrs, etc and a higher cost of credit without any reason.

Category: Credit Card Related


In [ ]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
   
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['Consumer complaint narrative'] = data['Consumer complaint narrative'].apply(clean_text)
     

In [ ]:
example_complain(10)

without provocation received notice credit line decreased nearly 100 available credit reduced rough amount available balance called question change provided nobdescript response referencing report understanding fcra entitled copy report refused citi given eplanation predatory affects utilization credit subjecting increase aprs etc higher cost credit without reason

Category: Credit Card Related


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Consumer complaint narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 123551 unique tokens.


In [ ]:

X = tokenizer.texts_to_sequences(data['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (249830, 250)


In [ ]:
Y = pd.get_dummies(data['Product']).values
print('Shape of label tensor:', Y.shape)
     

Shape of label tensor: (249830, 4)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(224847, 250) (224847, 4)
(24983, 250) (24983, 4)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
     

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 5,080,804
Trainable params: 5,080,804
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
     

Epoch 1/5
3162/3162 [==============================] - 1615s 510ms/step - loss: 0.5743 - accuracy: 0.8022 - val_loss: 0.4529 - val_accuracy: 0.8564
Epoch 2/5
3162/3162 [==============================] - 1619s 512ms/step - loss: 0.5290 - accuracy: 0.8200 - val_loss: 0.5031 - val_accuracy: 0.8375
Epoch 3/5
3162/3162 [==============================] - 1612s 510ms/step - loss: 0.4352 - accuracy: 0.8529 - val_loss: 0.3805 - val_accuracy: 0.8718
Epoch 4/5
3162/3162 [==============================] - 1609s 509ms/step - loss: 0.3489 - accuracy: 0.8820 - val_loss: 0.3589 - val_accuracy: 0.8783
Epoch 5/5
3162/3162 [==============================] - 1643s 520ms/step - loss: 0.3146 - accuracy: 0.8930 - val_loss: 0.3446 - val_accuracy: 0.8824


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

781/781 [==============================] - 40s 51ms/step - loss: 0.3634 - accuracy: 0.8755
Test set
  Loss: 0.363
  Accuracy: 0.876


In [ ]:
new_complaint = ['credit card account HDFC,The following bank of HDFC visa credit card account do not belong to me']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Others Services','Credit Card Related','Loan Related','Bank Services']
print(pred, labels[np.argmax(pred)])

1/1 [==============================] - 0s 51ms/step
[[0.01104178 0.9800703  0.00231324 0.00657468]] Credit Card Related
